In [1]:
import copy
import numpy as np
import os
import torch
import torch.nn as nn
import torch.utils.data as utils
import torch.nn.functional as F
import higher
import pickle
from torchvision import datasets, transforms
import argparse
import tqdm

from dataloaders import get_data
from models import *
from sklearn.metrics import accuracy_score, precision_score

## The student model

Here we define the student model to be used to train on cifar-10 dataset. <br />
The output is the 10 dimensional softmax output

In [2]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.leaky_relu(F.max_pool2d(self.conv1(x), 2), negative_slope=0.1)
        x = F.leaky_relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2), negative_slope=0.1)
        x = x.view(-1, 320)
        x = F.leaky_relu(self.fc1(x), negative_slope=0.1)
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.softmax(x, dim=1)

sample_student = Net()
preds = sample_student(torch.randn((1,1, 28,28)))
print(torch.sum(preds))
del sample_student
del preds

tensor(1.0000, grad_fn=<SumBackward0>)


## Teacher Model

We define the teacher model which calculates the weighted cross entropy loss on <br />


In [16]:
class Teacher(nn.Module):
    def __init__(self, classes=10, st_dim=13, hidden_dim=16):
        super(Teacher, self).__init__()
        self.V = torch.nn.Parameter(torch.randn(hidden_dim,st_dim))
        self.V.requires_grad = True
        self.W = torch.nn.Parameter(torch.randn(classes,classes,hidden_dim))
        self.W.requires_grad = True
        self.phi = torch.nn.Parameter(0.1 * torch.randn(classes,classes) + torch.eye(classes))
        self.phi.requires_grad = True
        self.softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()
    def forward(self, st, pw, y):
        '''
        W_channel_weights = self.softmax(torch.matmul(st, self.V.T))
        phi = torch.matmul(self.W, W_channel_weights)
        logpw_weights = torch.matmul(y, phi)
        loss = torch.mean(-1 * self.sigmoid(logpw_weights*torch.log(pw)))
        '''
        loss = torch.mean(-1 * self.sigmoid(torch.matmul(y, self.phi) * torch.log(pw)))
        
        return loss

In [17]:
def get_loss(student, teacher, x, y, state=None, val=False):
    if val:
        # this is the loss function that the teacher tries to minimize
        return  -1 * torch.mean(torch.sum(y * student(x), dim=1))
    else:
        # this is the loss function given by the teacher to optimize the student 
        # in the inner meta loop
        preds = student(x)
        student_loss = teacher(state, preds, y)
        return student_loss

In [18]:
teacher = Teacher()
student = Net()

st = torch.randn((13)).float()
sample_input = torch.randn((5, 1, 28,28))
pw = student(sample_input)
y = F.one_hot(torch.randint(0, 10, (5,)), num_classes=10).float()

result = teacher(st, pw, y)
#result.backward()
loss = get_loss(student, teacher, sample_input, y , st, val=False)
loss.backward()
# print( "V mean grad", torch.mean(torch.abs(teacher.V.grad)), "V max grad", torch.max(torch.abs(teacher.V.grad)) )
print("phi mean grad", torch.mean(torch.abs(teacher.phi.grad)), "V max grad", torch.max(torch.abs(teacher.phi.grad)) )
print(get_loss(student, teacher, sample_input, y , st, val=True))

del student
del teacher

phi mean grad tensor(0.0053) V max grad tensor(0.0139)
tensor(-0.0966, grad_fn=<MulBackward0>)


In [19]:
def get_metrics(student, train_dl, val_dl):
    train_preds = []
    train_gt = []
    val_preds = []
    val_gt = []
    # running the loop for training dataset
    for i, (x,y) in enumerate(train_dl):
        y_pred = torch.max(F.softmax(student(x), dim=1), 1)[1]
        train_preds.append(y_pred)
        train_gt.append(y)

    # running the loop for validation dataset
    for i, (x,y) in enumerate(val_dl):
        y_pred = torch.max(F.softmax(student(x), dim=1), 1)[1]
        val_preds.append(y_pred)
        val_gt.append(y)

    train_preds = torch.cat(train_preds, dim=0).detach().numpy()
    train_gt = torch.cat(train_gt, dim=0).detach().numpy()
    val_preds = torch.cat(val_preds, dim=0).detach().numpy()
    val_gt = torch.cat(val_gt, dim=0).detach().numpy()

    train_acc = accuracy_score(train_gt, train_preds, normalize=True)
    val_acc = accuracy_score(val_gt, val_preds, normalize=True)
    prescision = precision_score(val_gt, val_preds, average=None)
    return torch.cat([torch.tensor([train_acc]), torch.tensor([val_acc]), torch.tensor(prescision)],0)

In [20]:
train_loader, val_loader, test_loader = get_data(dataset="mnist")

In [21]:
def do_train_student(student, optimizer, teacher, train_dl, val_dl, iter=0,inner_steps=100):
    student.train()
    teacher.train()
    metrics = None# get_metrics(student, train_dl, val_dl).float()

    for i, (x,y) in enumerate(train_dl):
        x = x.float()
        y = F.one_hot(y, num_classes=10).float()

        # try to get the state vector here
        state = None # torch.cat([torch.tensor([float(iter/20.0)]), metrics])

        student_loss = get_loss(student, teacher, x, y, state=state, val=False)
        loss = student_loss
        if torch.isnan(student_loss):
            if torch.sum(student(x) <= 0):
                print("non positive value inside log found hence nan loss")
            print("=" * 30)
        # higher library exposes this interface for zero_grad on optimizer, loss.backward(), and optim.step()
        else:
            optimizer.step(loss)

        print('Train student: Step ', i, "loss value is", student_loss)
        if i>100:
            break
    return student

In [22]:
def do_train_teacher(teacher, optimizer, student, train_dl):
    lossdict = {
        'teach_loss' : [],
    }

    student.train()
    teacher.train()
    netloss = None
    for i, (x,y) in enumerate(train_dl):
        x = x.float()
        y = F.one_hot(y, num_classes=10).float()
        # Set val=True to get unweighted loss
        teacher_loss = get_loss(student, teacher, x, y, i, val=True)

        # Teacher is trained to minimise the student's classification error loss after training
        if netloss is None:
            netloss = teacher_loss
        else:
            netloss += teacher_loss

        print('Train Teacher: Step ', i, "  Teacher loss this step is:", teacher_loss)
        if i>50:
            break

    # Accumulate losses and do the step after we process outer_steps number of batches.
    netloss = netloss/(i+1)
    netloss.backward(retain_graph=True)
    return lossdict, teacher

In [28]:
def train(train_dl, val_dl, test_dl, num_steps):
    student = Net()
    teacher = Teacher()

    teacher_optim = torch.optim.Adam(teacher.parameters(), lr=1e-2)

    stud_state_dict = copy.deepcopy(student.state_dict())

    for ep in range(num_steps):
        student.load_state_dict(stud_state_dict, strict=True)
        stud_optim = torch.optim.Adam(student.parameters(), lr=1e-4)

        teacher_optim.zero_grad()
        print("inside epoch before higher optim")
        # The outer loop optimiser (teacher optimiser) does not change the student params, so copy_initial_weights can be True or False
        with higher.innerloop_ctx(student, stud_optim, copy_initial_weights=False) as (fnet, diffopt):
            fnet = do_train_student(fnet, diffopt, teacher, train_dl, val_dl)
            # Note: do_train_student does not edit the .grad field of teacher parameters
            # Therefore, no need to clear teacher grads before do_train_teacher.
            teacher_ld, teacher = do_train_teacher(teacher, teacher_optim, fnet, val_dl)
            student.load_state_dict(fnet.state_dict())
            # print( "V mean grad", torch.mean(torch.abs(teacher.V.grad)), "V max grad", torch.max(torch.abs(teacher.V.grad)) )
            print("phi mean grad", torch.mean(torch.abs(teacher.phi.grad)), "V max grad", torch.max(torch.abs(teacher.phi.grad)) )

            grad = student.conv1.weight.grad
            print( "student conv1 mean grad", torch.mean(torch.abs(grad)), "student conv1 max grad", torch.max(torch.abs(grad)) )

        teacher_optim.step()
        print(teacher.phi)

        #stud_train_ld = update_lossdict(stud_train_ld, student_ld)
        #teacher_train_ld = update_lossdict(teacher_train_ld, teacher_ld)

        print("Teacher training step: ", ep)

        #tld = eval_student(student, test_dl)
        #stud_test_ld = update_lossdict(stud_test_ld, tld)
        #vld = eval_student(student, val_dl)
        #stud_val_ld = update_lossdict(stud_val_ld, vld)

        #train_ep = eval_student(student, train_dl)
        #stud_train_ld = update_lossdict(stud_train_ld, train_ep) 

    return teacher, student

In [29]:
teacher, student = train(train_loader, val_loader, test_loader, num_steps=1000)

inside epoch before higher optim
Train student: Step  0 loss value is tensor(-0.4621, grad_fn=<MeanBackward0>)
Train student: Step  1 loss value is tensor(-0.4624, grad_fn=<MeanBackward0>)
Train student: Step  2 loss value is tensor(-0.4644, grad_fn=<MeanBackward0>)
Train student: Step  3 loss value is tensor(-0.4629, grad_fn=<MeanBackward0>)
Train student: Step  4 loss value is tensor(-0.4674, grad_fn=<MeanBackward0>)
Train student: Step  5 loss value is tensor(-0.4567, grad_fn=<MeanBackward0>)
Train student: Step  6 loss value is tensor(-0.4594, grad_fn=<MeanBackward0>)
Train student: Step  7 loss value is tensor(-0.4676, grad_fn=<MeanBackward0>)
Train student: Step  8 loss value is tensor(-0.4637, grad_fn=<MeanBackward0>)
Train student: Step  9 loss value is tensor(-0.4643, grad_fn=<MeanBackward0>)
Train student: Step  10 loss value is tensor(-0.4618, grad_fn=<MeanBackward0>)
Train student: Step  11 loss value is tensor(-0.4603, grad_fn=<MeanBackward0>)
Train student: Step  12 loss 

Train Teacher: Step  16   Teacher loss this step is: tensor(-0.1345, grad_fn=<MulBackward0>)
Train Teacher: Step  17   Teacher loss this step is: tensor(-0.1131, grad_fn=<MulBackward0>)
Train Teacher: Step  18   Teacher loss this step is: tensor(-0.0932, grad_fn=<MulBackward0>)
Train Teacher: Step  19   Teacher loss this step is: tensor(-0.1008, grad_fn=<MulBackward0>)
Train Teacher: Step  20   Teacher loss this step is: tensor(-0.1392, grad_fn=<MulBackward0>)
Train Teacher: Step  21   Teacher loss this step is: tensor(-0.0823, grad_fn=<MulBackward0>)
Train Teacher: Step  22   Teacher loss this step is: tensor(-0.1153, grad_fn=<MulBackward0>)
Train Teacher: Step  23   Teacher loss this step is: tensor(-0.1453, grad_fn=<MulBackward0>)
Train Teacher: Step  24   Teacher loss this step is: tensor(-0.1420, grad_fn=<MulBackward0>)
Train Teacher: Step  25   Teacher loss this step is: tensor(-0.0935, grad_fn=<MulBackward0>)
Train Teacher: Step  26   Teacher loss this step is: tensor(-0.0967, g

Train student: Step  41 loss value is tensor(-0.4645, grad_fn=<MeanBackward0>)
Train student: Step  42 loss value is tensor(-0.4628, grad_fn=<MeanBackward0>)
Train student: Step  43 loss value is tensor(-0.4586, grad_fn=<MeanBackward0>)
Train student: Step  44 loss value is tensor(-0.4637, grad_fn=<MeanBackward0>)
Train student: Step  45 loss value is tensor(-0.4586, grad_fn=<MeanBackward0>)
Train student: Step  46 loss value is tensor(-0.4647, grad_fn=<MeanBackward0>)
Train student: Step  47 loss value is tensor(-0.4583, grad_fn=<MeanBackward0>)
Train student: Step  48 loss value is tensor(-0.4600, grad_fn=<MeanBackward0>)
Train student: Step  49 loss value is tensor(-0.4661, grad_fn=<MeanBackward0>)
Train student: Step  50 loss value is tensor(-0.4596, grad_fn=<MeanBackward0>)
Train student: Step  51 loss value is tensor(-0.4662, grad_fn=<MeanBackward0>)
Train student: Step  52 loss value is tensor(-0.4664, grad_fn=<MeanBackward0>)
Train student: Step  53 loss value is tensor(-0.4592

Train Teacher: Step  42   Teacher loss this step is: tensor(-0.1007, grad_fn=<MulBackward0>)
Train Teacher: Step  43   Teacher loss this step is: tensor(-0.1058, grad_fn=<MulBackward0>)
Train Teacher: Step  44   Teacher loss this step is: tensor(-0.1019, grad_fn=<MulBackward0>)
Train Teacher: Step  45   Teacher loss this step is: tensor(-0.1006, grad_fn=<MulBackward0>)
Train Teacher: Step  46   Teacher loss this step is: tensor(-0.1008, grad_fn=<MulBackward0>)
Train Teacher: Step  47   Teacher loss this step is: tensor(-0.1091, grad_fn=<MulBackward0>)
Train Teacher: Step  48   Teacher loss this step is: tensor(-0.1178, grad_fn=<MulBackward0>)
Train Teacher: Step  49   Teacher loss this step is: tensor(-0.0843, grad_fn=<MulBackward0>)
Train Teacher: Step  50   Teacher loss this step is: tensor(-0.0953, grad_fn=<MulBackward0>)
Train Teacher: Step  51   Teacher loss this step is: tensor(-0.1445, grad_fn=<MulBackward0>)
phi mean grad tensor(0.0083) V max grad tensor(0.0224)
student conv1 m

Train student: Step  78 loss value is tensor(-0.4680, grad_fn=<MeanBackward0>)
Train student: Step  79 loss value is tensor(-0.4609, grad_fn=<MeanBackward0>)
Train student: Step  80 loss value is tensor(-0.4594, grad_fn=<MeanBackward0>)
Train student: Step  81 loss value is tensor(-0.4635, grad_fn=<MeanBackward0>)
Train student: Step  82 loss value is tensor(-0.4604, grad_fn=<MeanBackward0>)
Train student: Step  83 loss value is tensor(-0.4611, grad_fn=<MeanBackward0>)
Train student: Step  84 loss value is tensor(-0.4634, grad_fn=<MeanBackward0>)
Train student: Step  85 loss value is tensor(-0.4641, grad_fn=<MeanBackward0>)
Train student: Step  86 loss value is tensor(-0.4621, grad_fn=<MeanBackward0>)
Train student: Step  87 loss value is tensor(-0.4612, grad_fn=<MeanBackward0>)
Train student: Step  88 loss value is tensor(-0.4639, grad_fn=<MeanBackward0>)
Train student: Step  89 loss value is tensor(-0.4621, grad_fn=<MeanBackward0>)
Train student: Step  90 loss value is tensor(-0.4597

...

Train student: Step  4 loss value is tensor(-0.4635, grad_fn=<MeanBackward0>)
Train student: Step  5 loss value is tensor(-0.4707, grad_fn=<MeanBackward0>)
Train student: Step  6 loss value is tensor(-0.4717, grad_fn=<MeanBackward0>)
Train student: Step  7 loss value is tensor(-0.4558, grad_fn=<MeanBackward0>)
Train student: Step  8 loss value is tensor(-0.4625, grad_fn=<MeanBackward0>)
Train student: Step  9 loss value is tensor(-0.4618, grad_fn=<MeanBackward0>)
Train student: Step  10 loss value is tensor(-0.4704, grad_fn=<MeanBackward0>)
Train student: Step  11 loss value is tensor(-0.4691, grad_fn=<MeanBackward0>)
Train student: Step  12 loss value is tensor(-0.4642, grad_fn=<MeanBackward0>)
Train student: Step  13 loss value is tensor(-0.4657, grad_fn=<MeanBackward0>)
Train student: Step  14 loss value is tensor(-0.4757, grad_fn=<MeanBackward0>)
Train student: Step  15 loss value is tensor(-0.4704, grad_fn=<MeanBackward0>)
Train student: Step  16 loss value is tensor(-0.4642, grad

Train Teacher: Step  5   Teacher loss this step is: tensor(-0.1754, grad_fn=<MulBackward0>)
Train Teacher: Step  6   Teacher loss this step is: tensor(-0.1923, grad_fn=<MulBackward0>)
Train Teacher: Step  7   Teacher loss this step is: tensor(-0.1687, grad_fn=<MulBackward0>)
Train Teacher: Step  8   Teacher loss this step is: tensor(-0.1162, grad_fn=<MulBackward0>)
Train Teacher: Step  9   Teacher loss this step is: tensor(-0.1724, grad_fn=<MulBackward0>)
Train Teacher: Step  10   Teacher loss this step is: tensor(-0.1645, grad_fn=<MulBackward0>)
Train Teacher: Step  11   Teacher loss this step is: tensor(-0.1528, grad_fn=<MulBackward0>)
Train Teacher: Step  12   Teacher loss this step is: tensor(-0.1662, grad_fn=<MulBackward0>)
Train Teacher: Step  13   Teacher loss this step is: tensor(-0.1893, grad_fn=<MulBackward0>)
Train Teacher: Step  14   Teacher loss this step is: tensor(-0.1404, grad_fn=<MulBackward0>)
Train Teacher: Step  15   Teacher loss this step is: tensor(-0.1118, grad_f

Train student: Step  28 loss value is tensor(-0.4595, grad_fn=<MeanBackward0>)
Train student: Step  29 loss value is tensor(-0.4700, grad_fn=<MeanBackward0>)
Train student: Step  30 loss value is tensor(-0.4532, grad_fn=<MeanBackward0>)
Train student: Step  31 loss value is tensor(-0.4572, grad_fn=<MeanBackward0>)
Train student: Step  32 loss value is tensor(-0.4506, grad_fn=<MeanBackward0>)
Train student: Step  33 loss value is tensor(-0.4700, grad_fn=<MeanBackward0>)
Train student: Step  34 loss value is tensor(-0.4703, grad_fn=<MeanBackward0>)
Train student: Step  35 loss value is tensor(-0.4700, grad_fn=<MeanBackward0>)
Train student: Step  36 loss value is tensor(-0.4661, grad_fn=<MeanBackward0>)
Train student: Step  37 loss value is tensor(-0.4734, grad_fn=<MeanBackward0>)
Train student: Step  38 loss value is tensor(-0.4669, grad_fn=<MeanBackward0>)
Train student: Step  39 loss value is tensor(-0.4770, grad_fn=<MeanBackward0>)
Train student: Step  40 loss value is tensor(-0.4593

Train Teacher: Step  29   Teacher loss this step is: tensor(-0.2019, grad_fn=<MulBackward0>)
Train Teacher: Step  30   Teacher loss this step is: tensor(-0.1883, grad_fn=<MulBackward0>)
Train Teacher: Step  31   Teacher loss this step is: tensor(-0.1658, grad_fn=<MulBackward0>)
Train Teacher: Step  32   Teacher loss this step is: tensor(-0.1640, grad_fn=<MulBackward0>)
Train Teacher: Step  33   Teacher loss this step is: tensor(-0.2127, grad_fn=<MulBackward0>)
Train Teacher: Step  34   Teacher loss this step is: tensor(-0.1557, grad_fn=<MulBackward0>)
Train Teacher: Step  35   Teacher loss this step is: tensor(-0.1727, grad_fn=<MulBackward0>)
Train Teacher: Step  36   Teacher loss this step is: tensor(-0.1678, grad_fn=<MulBackward0>)
Train Teacher: Step  37   Teacher loss this step is: tensor(-0.1834, grad_fn=<MulBackward0>)
Train Teacher: Step  38   Teacher loss this step is: tensor(-0.1726, grad_fn=<MulBackward0>)
Train Teacher: Step  39   Teacher loss this step is: tensor(-0.1413, g

RuntimeError: python_error

In [34]:
print(teacher.phi)

NameError: name 'teacher' is not defined